# Fine Tuning Transformer for Summary Generation


### Introduction

In this tutorial we will be fine tuning a transformer model for **Summarization Task**. 
In this task a summary of a given article/document is generated when passed through a network. There are 2 types of summary generation mechanisms:

1. ***Extractive Summary:*** the network calculates the most important sentences from the article and gets them together to provide the most meaningful information from the article.
2. ***Abstractive Summary***: The network creates new sentences to encapsulate maximum gist of the article and generates that as output. The sentences in the summary may or may not be contained in the article. 

In this tutorial we will be generating ***Abstractive Summary***. 

#### Flow of the notebook

* As with all the tutorials previously, this notebook also follows a easy to follow steps. Making the process of fine tuning and training a Transformers model a straight forward task.
* However, unlike the other notebooks, in the tutorial, most of the sections have been created into functions, and they are called from the `main()` in the end of the notebook. 
* This is done to leverage the [Weights and Biases Service](https://www.wandb.com/) WandB in short.
* It is a experiment tracking, parameter optimization and artifact management service. That can be very easily integrated to any of the Deep learning or Machine learning frameworks. 

The notebook will be divided into separate sections to provide a organized walk through for the process used. This process can be modified for individual use cases. The sections are:

1. [Preparing Environment and Importing Libraries](#section01)
2. [Preparing the Dataset for data processing: Class](#section02)
3. [Fine Tuning the Model: Function](#section03)
4. [Validating the Model Performance: Function](#section04)
5. [Main Function](#section05)
    * [Initializing WandB](#section501)
    * [Importing and Pre-Processing the domain data](#section502)
    * [Creation of Dataset and Dataloader](#section503)
    * [Neural Network and Optimizer](#section504)
    * [Training Model and Logging to WandB](#section505)
    * [Validation and generation of Summary](#section506)
6. [Examples of the Summary Generated from the model](#section06)


#### Technical Details

This script leverages on multiple tools designed by other teams. Details of the tools used below. Please ensure that these elements are present in your setup to successfully implement this script.

- **Data**:
	- We are using the News Summary dataset available at [Kaggle](https://www.kaggle.com/sunnysai12345/news-summary)
	- This dataset is the collection created from Newspapers published in India, extracting, details that are listed below.  We are referring only to the first csv file from the data dump: `news_summary.csv`
	- There are`4514` rows of data.  Where each row has the following data-point:
		- **author** : Author of the article
		- **date** : Date the article was published
		- **headline**: Headline for the published article
		- **read_more** : URL for the article to follow online
		- **text**: This is the summary of the article
		- **ctext**: This is the complete article


- **Language Model Used**: 
    - This notebook uses one of the most recent and novel transformers model ***T5***. [Research Paper](https://arxiv.org/abs/1910.10683)    
    - ***T5*** in many ways is one of its kind transformers architecture that not only gives state of the art results in many NLP tasks, but also has a very radical approach to NLP tasks.
    - **Text-2-Text** - According to the graphic taken from the T5 paper. All NLP tasks are converted to a **text-to-text** problem. Tasks such as translation, classification, summarization and question answering, all of them are treated as a text-to-text conversion problem, rather than seen as separate unique problem statements.
    - **Unified approach for NLP Deep Learning** - Since the task is reflected purely in the text input and output, you can use the same model, objective, training procedure, and decoding process to ANY task. Above framework can be used for any task - show Q&A, summarization, etc. 
   - We will be taking inputs from the T5 paper to prepare our dataset prior to fine tuning and training.    
   - [Documentation for python](https://huggingface.co/transformers/model_doc/t5.html)

![**Each NLP problem as a “text-to-text” problem** - input: text, output: text](https://miro.medium.com/max/4006/1*D0J1gNQf8vrrUpKeyD8wPA.png) 
	 


- Hardware Requirements: 
	- Python 3.6 and above
	- Pytorch, Transformers and
	- All the stock Python ML Library
	- GPU enabled setup 
   

- **Script Objective**:
	- The objective of this script is to fine tune ***T5 *** to be able to generate summary, that a close to or better than the actual summary  while ensuring the important information from the article is not lost.

---
NOTE: 
We are using the Weights and Biases Tool-set in  this tutorial. The different components will be explained as we go through the article.

<a id='section01'></a>
### Preparing Environment and Importing Libraries

At this step we will be installing the necessary libraries followed by importing the libraries and modules needed to run our script. 
We will be installing:
* transformers
* wandb

Libraries imported are:
* Pandas
* Pytorch
* Pytorch Utils for Dataset and Dataloader
* Transformers
* T5 Model and Tokenizer
* wandb

Followed by that we will preapre the device for CUDA execeution. This configuration is needed if you want to leverage on onboard GPU. First we will check the GPU avaiable to us, using the nvidia command followed by defining our device.

Finally, we will be logging into the [wandb](https://www.wandb.com/) serice using the login command

In [2]:
!pip install transformers -q

# Code for TPU packages install
# !curl -q https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --apt-packages libomp5 libopenblas-dev

In [3]:
# Importing stock libraries
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

# Importing the T5 modules from huggingface/transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

/home/ren-admin/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# # Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

<a id='section02'></a>
### Preparing the Dataset for data processing: Class

We will start with creation of Dataset class - This defines how the text is pre-processed before sending it to the neural network. This dataset will be used the the Dataloader method that will feed  the data in batches to the neural network for suitable training and processing. 
The Dataloader and Dataset will be used inside the `main()`.
Dataset and Dataloader are constructs of the PyTorch library for defining and controlling the data pre-processing and its passage to neural network. For further reading into Dataset and Dataloader read the [docs at PyTorch](https://pytorch.org/docs/stable/data.html)

#### *CustomDataset* Dataset Class
- This class is defined to accept the Dataframe as input and generate tokenized output that is used by the **T5** model for training. 
- We are using the **T5** tokenizer to tokenize the data in the `text` and `ctext` column of the dataframe. 
- The tokenizer uses the ` batch_encode_plus` method to perform tokenization and generate the necessary outputs, namely: `source_id`, `source_mask` from the actual text and `target_id` and `target_mask` from the summary text.
- To read further into the tokenizer, [refer to this document](https://huggingface.co/transformers/model_doc/t5.html#t5tokenizer)
- The *CustomDataset* class is used to create 2 datasets, for training and for validation.
- *Training Dataset* is used to fine tune the model: **80% of the original data**
- *Validation Dataset* is used to evaluate the performance of the model. The model has not seen this data during training. 

#### Dataloader: Called inside the `main()`
- Dataloader is used to for creating training and validation dataloader that load data to the neural network in a defined manner. This is needed because all the data from the dataset cannot be loaded to the memory at once, hence the amount of data loaded to the memory and then passed to the neural network needs to be controlled.
- This control is achieved using the parameters such as `batch_size` and `max_len`.
- Training and Validation dataloaders are used in the training and validation part of the flow respectively

In [5]:
# Creating a custom dataset for reading the dataframe and loading it into the dataloader to pass it to the neural network at a later stage for finetuning the model and to prepare it for predictions

class CustomDataset(Dataset):

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

<a id='section03'></a>
### Fine Tuning the Model: Function

Here we define a training function that trains the model on the training dataset created above, specified number of times (EPOCH), An epoch defines how many times the complete data will be passed through the network. 

This function is called in the `main()`

Following events happen in this function to fine tune the neural network:
- The epoch, tokenizer, model, device details, testing_ dataloader and optimizer are passed to the `train ()` when its called from the `main()`
- The dataloader passes data to the model based on the batch size.
- `language_model_labels` are calculated from the `target_ids` also, `source_id` and `attention_mask` are extracted.
- The model outputs first element gives the loss for the forward pass. 
- Loss value is used to optimize the weights of the neurons in the network.
- After every 10 steps the loss value is logged in the wandb service. This log is then used to generate graphs for analysis. Such as [these](https://app.wandb.ai/abhimishra-91/transformers_tutorials_summarization?workspace=user-abhimishra-91)
- After every 500 steps the loss value is printed in the console.

In [6]:
# Creating the training function. This will be called in the main function. It is run depending on the epoch value.
# The model is put into train mode and then we wnumerate over the training loader and passed to the defined network 

def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in enumerate(loader, 0):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]

        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

<a id='section04'></a>
### Validating the Model Performance: Function

During the validation stage we pass the unseen data(Testing Dataset), trained model, tokenizer and device details to the function to perform the validation run. This step generates new summary for dataset that it has not seen during the training session. 

This function is called in the `main()`

This unseen data is the 20% of `news_summary.csv` which was seperated during the Dataset creation stage. 
During the validation stage the weights of the model are not updated. We use the generate method for generating new text for the summary. 

It depends on the `Beam-Search coding` method developed for sequence generation for models with LM head. 

The generated text and originally summary are decoded from tokens to text and returned to the `main()`

In [7]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

<a id='section05'></a>
### Main Function

The `main()` as the name suggests is the central location to execute all the functions/flows created above in the notebook. The following steps are executed in the `main()`:


<a id='section502'></a>
#### Importing and Pre-Processing the domain data

We will be working with the data and preparing it for fine tuning purposes. 
*Assuming that the `news_summary.csv` is already downloaded in your `data` folder*

* The file is imported as a dataframe and give it the headers as per the documentation.
* Cleaning the file to remove the unwanted columns.
* A new string is added to the main article column `summarize: ` prior to the actual article. This is done because **T5** had similar formatting for the summarization dataset. 
* The final Dataframe will be something like this:

|text|ctext|
|--|--|
|summary-1|summarize: article 1|
|summary-2|summarize: article 2|
|summary-3|summarize: article 3|

* Top 5 rows of the dataframe are printed on the console.

<a id='section503'></a>
#### Creation of Dataset and Dataloader

* The updated dataframe is divided into 80-20 ratio for test and validation. 
* Both the data-frames are passed to the `CustomerDataset` class for tokenization of the new articles and their summaries.
* The tokenization is done using the length parameters passed to the class.
* Train and Validation parameters are defined and passed to the `pytorch Dataloader contstruct` to create `train` and `validation` data loaders.
* These dataloaders will be passed to `train()` and `validate()` respectively for training and validation action.
* The shape of datasets is printed in the console.


<a id='section504'></a>
#### Neural Network and Optimizer

* In this stage we define the model and optimizer that will be used for training and to update the weights of the network. 
* We are using the `t5-base` transformer model for our project. You can read about the `T5 model` and its features above. 
* We use the `T5ForConditionalGeneration.from_pretrained("t5-base")` commad to define our model. The `T5ForConditionalGeneration` adds a Language Model head to our `T5 model`. The Language Model head allows us to generate text based on the training of `T5 model`.
* We are using the `Adam` optimizer for our project. This has been a standard for all our tutorials and is something that can be changed updated to see how different optimizer perform with different learning rates. 
* There is also a scope for doing more with Optimizer such a decay, momentum to dynamically update the Learning rate and other parameters. All those concepts have been kept out of scope for these tutorials. 


<a id='section506'></a>
#### Validation and generation of Summary

* After the training is completed, the validation step is initiated.
* As defined in the validation function, the model weights are not updated. We use the fine tuned model to generate new summaries based on the article text.
* An output is printed on the console giving a count of how many steps are complete after every 100th step. 
* The original summary and generated summary are converted into a list and returned to the main function. 
* Both the lists are used to create the final dataframe with 2 columns **Generated Summary** and **Actual Summary**
* The dataframe is saved as a csv file in the local drive.
* A qualitative analysis can be done with the Dataframe. 

In [8]:
TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
TRAIN_EPOCHS = 2        # number of epochs to train (default: 10)
VAL_EPOCHS = 1 
LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
SEED = 42               # random seed (default: 42)
MAX_LEN = 512
SUMMARY_LEN = 150 

# Set random seeds and deterministic pytorch for reproducibility
torch.manual_seed(SEED) # pytorch random seed
np.random.seed(SEED) # numpy random seed

# tokenzier for encoding the text
tokenizer = T5Tokenizer.from_pretrained("t5-base")


# Importing and Pre-Processing the domain data
# Selecting the needed columns only. 
# Adding the summarzie text in front of the text. This is to format the dataset similar to how T5 model was trained for summarization task. 
df = pd.read_csv('/home/ren-admin/paradigm/dash/nlg/news_summary.csv',encoding='latin-1')
df = df[['text','ctext']]
df.ctext = 'summarize: ' + df.ctext
print(df.head())


# Creation of Dataset and Dataloader
# Defining the train size. So 80% of the data will be used for training and the rest will be used for validation. 
train_size = 0.8
train_dataset=df.sample(frac=train_size,random_state = SEED)
val_dataset=df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(val_dataset.shape))


# Creating the Training and Validation dataset for further creation of Dataloader
training_set = CustomDataset(train_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)
val_set = CustomDataset(val_dataset, tokenizer, MAX_LEN, SUMMARY_LEN)

# Defining the parameters for creation of dataloaders
train_params = {
    'batch_size': TRAIN_BATCH_SIZE,
    'shuffle': True,
    'num_workers': 0
    }

val_params = {
    'batch_size': VALID_BATCH_SIZE,
    'shuffle': False,
    'num_workers': 0
    }

# Creation of Dataloaders for testing and validation. This will be used down for training and validation stage for the model.
training_loader = DataLoader(training_set, **train_params)
val_loader = DataLoader(val_set, **val_params)



# Defining the model. We are using t5-base model and added a Language model layer on top for generation of Summary. 
# Further this model is sent to device (GPU/TPU) for using the hardware.
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = model.to(device)

# Defining the optimizer that will be used to tune the weights of the network in the training session. 
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

# Training loop
print('Initiating Fine-Tuning for the model on our dataset')

for epoch in range(TRAIN_EPOCHS):
    train(epoch, tokenizer, model, device, training_loader, optimizer)


# Validation loop and saving the resulting file with predictions and actuals in a dataframe.
# Saving the dataframe as predictions.csv
print('Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe')
for epoch in range(VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})
    final_df.to_csv('/home/ren-admin/paradigm/dash/nlg/predictions.csv')
    print('Output Files generated for review')

/home/ren-admin/miniconda3/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


                                                text  \
0  The Administration of Union Territory Daman an...   
1  Malaika Arora slammed an Instagram user who tr...   
2  The Indira Gandhi Institute of Medical Science...   
3  Lashkar-e-Taiba's Kashmir commander Abu Dujana...   
4  Hotels in Maharashtra will train their staff t...   

                                               ctext  
0  summarize: The Daman and Diu administration on...  
1  summarize: From her special numbers to TV?appe...  
2  summarize: The Indira Gandhi Institute of Medi...  
3  summarize: Lashkar-e-Taiba's Kashmir commander...  
4  summarize: Hotels in Mumbai and other Indian c...  
FULL Dataset: (4514, 2)
TRAIN Dataset: (3611, 2)
TEST Dataset: (903, 2)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Initiating Fine-Tuning for the model on our dataset


/home/ren-admin/miniconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Epoch: 0, Loss:  8.259500503540039
Epoch: 0, Loss:  1.472969889640808
Epoch: 0, Loss:  1.9409586191177368
Epoch: 0, Loss:  1.289207100868225
Epoch: 1, Loss:  1.085049033164978
Epoch: 1, Loss:  1.3684293031692505
Epoch: 1, Loss:  1.399253010749817
Epoch: 1, Loss:  1.1674277782440186
Now generating summaries on our fine tuned model for the validation dataset and saving it in a dataframe
Completed 0
Completed 100
Completed 200
Completed 300
Completed 400
Output Files generated for review


In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import time
from transformers import pipeline
import datetime
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests
import urllib
import fitz
import re

pdf = "/home/ren-admin/paradigm/dash/wasde/wasde.pdf"
doc = fitz.open(pdf)
stop = 0
for i in range(doc.page_count):
    contents = doc.load_page(i).get_text("text")
    if "Approved by the" in contents and i != 0:
        stop = i
        break
allgrains = ""
for j in range(stop+1):
    contents = doc.load_page(j).get_text("text")
    allgrains = allgrains + contents
seven = re.split('WHEAT: |COARSE GRAINS: |RICE: |OILSEEDS: |SUGAR: |LIVESTOCK, POULTRY, AND DAIRY: |COTTON: ', allgrains)
seven = seven[1:]
remove = ["\nWASDE-626-2 \n \n \n", "\nWASDE-626-3 \n \n \n", "\nWASDE-626-4 \n \n \n", "\nWASDE-626-5 \n \n \n"]
for i in range(len(seven)):
    for j in range(4):
            if remove[j] in seven[i]:
                seven[i] = seven[i].replace(remove[j], "")
for i in range(len(seven)):
    seven[i] = seven[i].split("\n \n")[:-1]
for i in range(len(seven)):
    for j in range(len(seven[i])):
            seven[i][j] = seven[i][j].strip()
            seven[i][j] = seven[i][j].replace("\n", "")
            seven[i][j] = seven[i][j].replace("  ", " ")
for i in range(len(seven[-1])):
        if seven[-1][i] == '':
            break
seven[-1] = seven[-1][:i]

wheat = " ".join(seven[0])
coarse = " ".join(seven[1])
rice = " ".join(seven[2])
oilseed = " ".join(seven[3])
sugar = " ".join(seven[4])
meats = " ".join(seven[5])
cotton = " ".join(seven[6])

In [10]:
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizerp = PegasusTokenizer.from_pretrained(model_name)
modelp = PegasusForConditionalGeneration.from_pretrained(model_name)

wheat_sum = []
coarse_sum = []
rice_sum = []
oilseed_sum = []
sugar_sum = []
meats_sum = []
cotton_sum = []
for sen in seven[0]:
    input_ids = tokenizerp(sen, return_tensors="pt").input_ids
    output = modelp.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    wheat_sum.append(tokenizerp.decode(output[0], skip_special_tokens=True))

for sen in seven[1]:
    input_ids = tokenizerp(sen, return_tensors="pt").input_ids
    output = modelp.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    coarse_sum.append(tokenizerp.decode(output[0], skip_special_tokens=True))

for sen in seven[2]:
    input_ids = tokenizerp(sen, return_tensors="pt").input_ids
    output = modelp.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    rice_sum.append(tokenizerp.decode(output[0], skip_special_tokens=True))

for sen in seven[3]:
    input_ids = tokenizerp(sen, return_tensors="pt").input_ids
    output = modelp.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    oilseed_sum.append(tokenizerp.decode(output[0], skip_special_tokens=True))

for sen in seven[4]:
    input_ids = tokenizerp(sen, return_tensors="pt").input_ids
    output = modelp.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    sugar_sum.append(tokenizerp.decode(output[0], skip_special_tokens=True))

for sen in seven[5]:
    input_ids = tokenizerp(sen, return_tensors="pt").input_ids
    output = modelp.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    meats_sum.append(tokenizerp.decode(output[0], skip_special_tokens=True))

for sen in seven[6]:
    input_ids = tokenizerp(sen, return_tensors="pt").input_ids
    output = modelp.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
    cotton_sum.append(tokenizerp.decode(output[0], skip_special_tokens=True))

wheat_sum = " ".join(wheat_sum)
coarse_sum = " ".join(coarse_sum)
rice_sum = " ".join(rice_sum)
oilseed_sum = " ".join(oilseed_sum)
sugar_sum = " ".join(sugar_sum)
meats_sum = " ".join(meats_sum)
cotton_sum = " ".join(cotton_sum)

In [20]:
print(cotton)

This month’s 2022/23 U.S. cotton supply and demand forecasts are unchanged relative to last month. The projected marketing year average price received by producers is also unchanged at 83 cents per pound. The global 2022/23 cotton supply and demand forecasts this month include lower consumption and trade, and higher production and stocks. Beginning stocks are almost 900,000 bales higher as historical consumption estimates for China and Uzbekistan are updated to align with data from official and other sources. World cotton consumption in 2022/23 is 555,000 bales lower this month with reductions in Turkey, Pakistan, Indonesia, and Bangladesh. Projected imports are lower for each of these countries—and for China—while exports are lower for Brazil, India, and Argentina, with world trade totaling 785,000 bales lower. Production is more than 700,000 bales higher as larger expected crops in China, Australia, and Uzbekistan more than offset reduced prospects for India. At 91.1 million bales, 2

In [21]:
# Pegasus
print(cotton_sum)

Supply and demand forecasts for 2022/23 unchanged from last month. projected marketing year average price unchanged at 83 cents per pound Lower consumption and trade, higher production and stocks. Global ending stocks projected at 91.1 million bales


In [22]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
tokenizer2 = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")
model2 = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-summarize-news")

def summarize(text, max_length=300):
    input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True).to(device='cuda')
    generated_ids = model.generate(input_ids=input_ids, num_beams=2, max_length=max_length, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
    preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    return preds[0]

def summarize2(text, max_length=300):
    input_ids = tokenizer2.encode(text, return_tensors="pt", add_special_tokens=True)
    generated_ids = model2.generate(input_ids=input_ids, num_beams=2, max_length=max_length, repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
    preds = [tokenizer2.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
    return preds[0]

# untrained T5
print(summarize2(cotton))

world trade totalling 785,000 bales lower. Beginning stocks are almost 900,000 bales higher as historical consumption estimates for China and Uzbekistan are updated to align with data from official and other sources. Cotton production is more than 700,000 bales higher as larger expected crops in China, Australia, and Uzbekistan offset reduced prospects for India. Cotton production is almost 900,000 bales higher. World cotton consumption in 2022/23 is 555,000 bales lower. Projected


In [14]:
# trained T5
print(summarize(cotton))

is 83 cents per pound. World cotton consumption in 2022/23 is 555,000 bales lower this month with reductions in Turkey, Pakistan, Indonesia, and Bangladesh. Beginning stocks are almost 900,000 bales higher as larger expected crops in China, Australia, and Uzbekistan offset reduced prospects for India.


In [15]:
from summarizer import Summarizer, TransformerSummarizer
bert_model = Summarizer()
bert_summary = ''.join(bert_model(cotton, min_length=60))
print(bert_summary)

2023-03-29 16:52:31.157761: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-29 16:52:31.158528: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-29 16:52:31.159052: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-29 16:52:31.159557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-29 16:52:31.499213: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
This month’s 2022/23 U.S. cotton supply and demand forecasts are unchanged relative to last month. The global 2022/23 cotton supply and demand forecasts this month include lower consumption and trade, and higher production and stocks.


In [16]:
GPT2_model = TransformerSummarizer(transformer_type="GPT2", transformer_model_key="gpt2-medium")
full = ''.join(GPT2_model(cotton, min_length=60))
print(full)

This month’s 2022/23 U.S. cotton supply and demand forecasts are unchanged relative to last month. Beginning stocks are almost 900,000 bales higher as historical consumption estimates for China and Uzbekistan are updated to align with data from official and other sources.


In [17]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

flan_model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
flan_tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

f_input_text = "A step by step recipe to make bolognese pasta:"
f_input_ids = flan_tokenizer(f_input_text, return_tensors="pt").input_ids

f_outputs = flan_model.generate(f_input_ids)
print(flan_tokenizer.decode(f_outputs[0], skip_special_tokens=True))

/home/ren-admin/miniconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Preheat the oven to 375 degrees F. In a large bowl, combine the olive


In [18]:
f_inputs = flan_tokenizer("A step by step recipe to make bolognese pasta:", return_tensors="pt")
f_outputs = flan_model.generate(**f_inputs)
print(flan_tokenizer.batch_decode(f_outputs, skip_special_tokens=True))

['Preheat the oven to 375 degrees F. In a large bowl, combine the olive']


In [19]:
command = "Abstractively summarize: " + cotton
flan_inputs = tokenizer(command, return_tensors="pt")
flan_outputs = flan_model.generate(**flan_inputs)
print(flan_tokenizer.batch_decode(flan_outputs, skip_special_tokens=True))

['U.S. cotton production is expected to be boosted by lower global cotton consumption and lower']
